-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [1]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm  
import requests 

import os
os.chdir("c:\\Users\\jirka\\Documents\\MyProjects\\Sreality")
from scraper.scraper import SrealityScraper
from db_managment.data_manager import DataManager
from utils.utils import Utilities


In [2]:
utils = Utilities()
timestamp = utils.generate_timestamp()
timestamp

'2024-04-18 09:59:44'

### My regular run

In [ ]:
timestamp = "20240418_0948"
scraper = SrealityScraper()

data_prodej_byty = scraper.run_scraping(category_main_cb=1,category_type_cb=1)
df_data_prodej_byty = pd.DataFrame(data_prodej_byty)
scraper.safe_save_csv(df_data_prodej_byty, f"data_{timestamp}")

data_all = scraper.run_scraping()
df_data_all = pd.DataFrame(data_all)
scraper.safe_save_csv(df_data_all, f"data_all_{timestamp}")

In [ ]:
# Just testing count
with requests.Session() as session:
    result =  session.request(method="GET", url="https://www.sreality.cz/api/cs/v2/estates/count").json()
result

In [ ]:
# Just testing
url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
with requests.Session() as session:
    result =  session.request(method="GET", url=url_filter).json()
result

## Table managment test

In [ ]:
from db_managment.data_manager import DataManager
import pandas as pd

data_manager = DataManager()

In [ ]:
data_manager.get_all_rows("scraped_prices")

In [ ]:
data_manager.clear_table("scraped_prices")

In [ ]:
data_manager.drop_table("scraped_prices")

In [ ]:
data_manager.create_table("scraped_prices")

In [ ]:
upload = pd.DataFrame(data = [["2024-15-04 12:00:01"]],
                      columns=["timestamp"])
data_manager.insert_new_batch(upload)

In [ ]:

upload = pd.DataFrame(data = [["2109527372",
                              "Prodej bytu 3+1 69 m²",
                              "1",
                              "1",
                              "7",
                              "byt",
                              "prodej",
                              "3+1",
                              "69.0",
                              "brno-zabovresky-lucni",
                              "jihomoravský kraj",
                              "Brno",
                              "49.208683",
                              "16.58177716",
                              "2024-04-13 10:34:53",
                              "12345000",
                              "2024-04-16 21:22:23"
                              ]],
                      columns=["code", "name", 
                               "category_main_cb", "category_type_cb", "category_sub_cb",
                               "type_of_building", "type_of_deal", "type_of_rooms", 
                               "size_meters", "locality",
                               "region", "district", "latitude",
                               "longitude", "timestamp", "price", "created_at"])
data_manager.insert_new_estate(upload)

In [ ]:

#TODO: how about unit test for this?
#? should fail predictably as it breakes foreign key constraints i made:)
#upload = pd.DataFrame(data = [["911", "4242", "12345000"]],
#                      columns=["estate_id", "batch_id", "price"])

upload = pd.DataFrame(data = [["1", "1", "12345000", "2024-04-16 21:22:23"]],
                      columns=["estate_id", "batch_id", "price", "created_at"])
data_manager.insert_new_price(upload)

## Processing new offers

In [ ]:
from db_managment.data_manager import DataManager
import pandas as pd

data_manager = DataManager()

In [ ]:
df_new = pd.read_csv("data_2024/data_20240418_0045.csv", sep=";", encoding="utf-8")
if 'Unnamed: 0' in df_new.columns:
    df_new.drop('Unnamed: 0', axis=1, inplace=True)

print(len(df_new))
display(df_new.tail(2))

In [ ]:
new_estates = data_manager.filter_new_estates(df_new)

print(len(new_estates))
display(new_estates.tail(2))

In [ ]:
print(new_estates.loc[0, "category_main_cb"])
print(data_manager.translate_type_of_building(new_estates.loc[0, "category_main_cb"]))

print(new_estates.loc[0, "category_type_cb"])
print(data_manager.translate_type_of_deal(new_estates.loc[0, "category_main_cb"]))

print(new_estates.loc[0, "category_sub_cb"])
print(data_manager.translate_type_of_rooms(new_estates.loc[0, "category_main_cb"]))

worfklow new price, pokud neexistuje Estate, resp batch
- load csv s offers
- filter_new_estates - najde mi ty, jejichž code ještě není v tabulce estate_details
- přeloží to pro ně typ budovy, typ  dealu, počet pokojů - z dictionary
- před zápisem do DB vygeneruje timestamp 
- insertuje se new_batch - pokud taky neexistuje !!
- jinak se může insertovat new new_estate
- a pak klidně rovnou i new price, navázané na tuto batch a price